In [ ]:
!pip install simcse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from simcse import SimCSE
import random
import csv
import numpy as np
from tqdm import tqdm

df = pd.read_csv('/content/combined.csv')

train, test = train_test_split(df, test_size=0.8)

model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

In [ ]:
# Calcoliamo gli encoding che ci serviranno in seguito
train['text_embedding'] = list(model.encode(train['text'].tolist()))
train['ade_embedding'] = list(model.encode(train['ade'].tolist()))
train['meddra_embedding'] = list(model.encode(train['meddra'].tolist()))


100%|██████████| 23/23 [00:39<00:00,  1.73s/it]


In [ ]:
texts = []
ades = []
meddras = []
positives = []
positive_ades = []
positive_meddras = []
negatives = []
negative_ades = []
negative_meddras = []

# Per lo stesso testo 5 esempi positivi e 5 negativi
# Alla peggio sono gli stessi 5 esempi
N_PAIRS_FOR_TEXT = 5

for _ in tqdm(range(N_PAIRS_FOR_TEXT)):
  # Calcolo dei positives e dei negatives
  for index, (text, ade, meddra, _, text_embedding, _, _) in train.iterrows():
      same_meddra_pair = train.loc[(train['meddra'] == meddra) & (train['text'] != text)]
      # Una riga con lo stesso , meddra allora l'esempio positivo è il meddra
      if not same_meddra_pair.empty:
        same_meddra_row = same_meddra_pair.sample(n=1) 
        positive: str = same_meddra_row.iloc[0, train.columns.get_loc('text')]
        positive_ade: str = same_meddra_row.iloc[0, train.columns.get_loc('ade')]
        positive_meddra: str = same_meddra_row.iloc[0, train.columns.get_loc('meddra')]
      else:
        positive: str = meddra
        positive_ade: str = ade
        positive_meddra: str = meddra

      # Opzione 1: L'esempio negativo è semplicemente un testo con meddra diverso
      # I testi che hanno lo stesso meddra e che non possiamo prendere
      rows_with_same_meddra = train.loc[train['meddra'] == meddra]
      texts_with_same_meddra = rows_with_same_meddra['text'].tolist()
      # Dobbiamo fare così perchè potrebbe esserci un testo con più ade che mappano in diversi meddra per cui rischiamo
      # di prendere come esempio negativo un testo con lo stesso meddra ad esempio
      # testo1 ade1 meddra1
      # testo1 ade2 meddra2
      # In questo caso se volessimo trovare un esempio negativo per meddra2 potremmo prendere la riga 1
      # selezionando il testo1 che ha un meddra diverso (che però contiene anche lo stesso meddra2) potremmo quindi
      # addirittura selezionare lo stesso testo come esempio negativo
      different_meddra_rows = train.loc[~ train['text'].isin(texts_with_same_meddra)]
      # Opzione 2: Usiamo il framework SimCSE per cercare frasi con embedding simili al testo con diverso meddra
      if not different_meddra_rows.empty:
        # Troviamo l'indice dell'embedding più vicino
        diff_rows_sample = different_meddra_rows.sample(n=N_PAIRS_FOR_TEXT)
        different_meddra_embeddings = np.array([embedding.numpy() for embedding in diff_rows_sample['text_embedding']])

        distances = np.linalg.norm(different_meddra_embeddings - text_embedding.numpy(), axis=1)
        min_distance_index = np.argmin(distances)

        # Con l'indice prendiamo la riga nel vettore dei testi con meddra diverso
        negative =  diff_rows_sample.iloc[min_distance_index, train.columns.get_loc('text')]
        negative_ade: str =  diff_rows_sample.iloc[min_distance_index, train.columns.get_loc('ade')]
        negative_meddra: str =  diff_rows_sample.iloc[min_distance_index, train.columns.get_loc('meddra')]

      else:
        continue
        

      texts.append(text)
      ades.append(ade)
      meddras.append(meddra)
      positives.append(positive)
      positive_ades.append(positive_ade)
      positive_meddras.append(positive_meddra)
      negatives.append(negative)
      negative_ades.append(negative_ade)
      negative_meddras.append(negative_meddra)

100%|██████████| 5/5 [00:20<00:00,  4.04s/it]


In [ ]:
positive_negative_pairs = pd.DataFrame({
    'text': texts,
    'positive': positives,
    'negative': negatives,
    'ade': ades,
    'meddra': meddras,
    'positive_ade': positive_ades,
    'positive_meddra': positive_meddras,
    'negative_ade': negative_ades,
    'negative_meddra': negative_meddras
})

print(positive_negative_pairs.head(5))


                                                text  \
0  sterted with swelling in one ankle then a knee...   
1  weakness, fatigue, muscle spasms, stiffness al...   
2  upset stomach, nausea, vomiting, abdominal pai...   
3  last night i was a mermaid and then a wizard w...   
4  after a routine check up my cholesterol level ...   

                                            positive  \
0                          constant lightheadedness.   
1  severe back pain muscles pain loss of energy, ...   
2  rt @uffelanie: just wondering where the side e...   
3  @droz i take trazodone for my #insomina &amp; ...   
4    i have been on this medicine for about 8 years.   

                                            negative  \
0  my feet feel like i have stone bruises just in...   
1  bloating - gi difficulties, added weight, weak...   
2  shoulder and neck muscle pain, tender joints, ...   
3  imence pain in legs,calf,feet,cramping in legs...   
4  i had diarrhea for a whole week;bad uterine

In [ ]:
positive_negative_pairs.to_csv('/content/positive_negative_pairs_train.csv', index=False)
test.to_csv('/content/positive_negative_pairs_test.csv', index=False)

Scarichiamo i file appena creati

In [ ]:
#from google.colab import files
#files.download('/content/positive_negative_pairs_train.csv')
#files.download('/content/positive_negative_pairs_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>